# Arduino with Python 

## Uploading the Firmata Sketch
Before you write your Python program to drive Arduino, you have to upload the Firmata sketch so that you can use that protocol to control the board. The sketch is available in the Arduino IDE’s built-in examples. To open it, access the File menu, then Examples, followed by Firmata, and finally StandardFirmata:

![Image](./arduino_ide_firmdata.jpg)

The sketch will be loaded into a new IDE window. To upload it to the Arduino, you can follow the same steps you did before:

Plug the USB cable into the PC.
Select the appropriate board and port on the IDE.
Press Upload.
After the upload is finished, you won’t notice any activity on the Arduino. To control it, you still need a program that can communicate with the board through the serial connection. To work with the Firmata protocol in Python, you’ll need the pyFirmata package, which you can install with pip:


In [4]:
! pip install pyfirmata

After the installation finishes, you can run an equivalent Blink application using Python and Firmata:

In [3]:
import pyfirmata
import time

board = pyfirmata.Arduino('/dev/ttyACM0')

while True:
    board.digital[13].write(1)
    time.sleep(1)
    board.digital[13].write(0)
    time.sleep(1)

Here’s how this program works. You import pyfirmata and use it to establish a serial connection with the Arduino board, which is represented by the board object in line 4. You also configure the port in this line by passing an argument to pyfirmata.Arduino(). You can use the Arduino IDE to find the port.

board.digital is a list whose elements represent the digital pins of the Arduino. These elements have the methods read() and write(), which will read and write the state of the pins. Like most embedded device programs, this program mainly consists of an infinite loop:

In line 7, digital pin 13 is turned on, which turns the LED on for one second.
In line 9, this pin is turned off, which turns the LED off for one second.
Now that you know the basics of how to control an Arduino with Python, let’s go through some applications to interact with its inputs and outputs.

## Reading Digital Inputs

Digital inputs can have only two possible values. In a circuit, each of these values is represented by a different voltage. The table below shows the digital input representation for a standard Arduino Uno board:

Value	Level	Voltage
0	Low	0V
1	High	5V
To control the LED, you’ll use a push button to send digital input values to the Arduino. The button should send 0V to the board when it’s released and 5V to the board when it’s pressed.

![circuit1](./circuit1.jpg)

Now that you’ve assembled the circuit, you have to run a program on the PC to control it using Firmata. This program will turn on the LED, based on the state of the push button:

In [3]:
import pyfirmata
import time

board = pyfirmata.Arduino('/dev/ttyACM0')

it = pyfirmata.util.Iterator(board)
it.start()

board.digital[10].mode = pyfirmata.INPUT

while True:
    sw = board.digital[10].read()
    if sw is True:
        board.digital[13].write(1)
    else:
        board.digital[13].write(0)
    time.sleep(0.1)

## Let’s walk through this program:

Lines 1 and 2 import pyfirmata and time.
Line 4 uses pyfirmata.Arduino() to set the connection with the Arduino board.
Line 6 assigns an iterator that will be used to read the status of the inputs of the circuit.
Line 7 starts the iterator, which keeps a loop running in parallel with your main code. The loop executes board.iterate() to update the input values obtained from the Arduino board.
Line 9 sets pin 10 as a digital input with pyfirmata.INPUT. This is necessary since the default configuration is to use digital pins as outputs.
Line 11 starts an infinite while loop. This loop reads the status of the input pin, stores it in sw, and uses this value to turn the LED on or off by changing the value of pin 13.
Line 17 waits 0.1 seconds between iterations of the while loop. This isn’t strictly necessary, but it’s a nice trick to avoid overloading the CPU, which reaches 100% load when there isn’t a wait command in the loop.


## Reading Analog Inputs

In contrast to digital inputs, which can only be on or off, analog inputs are used to read values in some range. On the Arduino Uno, the voltage to an analog input ranges from 0V to 5V. Appropriate sensors are used to measure physical quantities, such as distances. These sensors are responsible for encoding these physical quantities in the proper voltage range so they can be read by the Arduino.

To read an analog voltage, the Arduino uses an analog-to-digital converter (ADC), which converts the input voltage to a digital number with a fixed number of bits. This determines the resolution of the conversion. The Arduino Uno uses a 10-bit ADC and can determine 1024 different voltage levels.

The voltage range for an analog input is encoded to numbers ranging from 0 to 1023. When 0V is applied, the Arduino encodes it to the number 0. When 5V is applied, the encoded number is 1023. All intermediate voltage values are proportionally encoded.

A potentiometer is a variable resistor that you can use to set the voltage applied to an Arduino analog input. You’ll connect it to an analog input to control the frequency of a blinking LED:

In this circuit, the LED is set up just as before. The end terminals of the potentiometer are connected to ground (GND) and 5V pins. This way, the central terminal (the cursor) can have any voltage in the 0V to 5V range depending on its position, which is connected to the Arduino on analog pin A0.

Using a breadboard, you can assemble this circuit as follows:

![curcuit2](./circuit2.jpg)

In [3]:
import pyfirmata
import time

board = pyfirmata.Arduino('/dev/ttyACM0')
it = pyfirmata.util.Iterator(board)
it.start()

analog_input = board.get_pin('a:0:i')

while True:
    analog_value = analog_input.read()
    print(analog_value)
    time.sleep(0.1)

In line 8, you set up analog_input as the analog A0 input pin with the argument 'a:0:i'. Inside the infinite while loop, you read this value, store it in analog_value, and display the output to the console with print(). When you move the potentiometer while the program runs, you should output similar to this:

0.0
0.0293
0.1056
0.1838
0.2717
0.3705
0.4428
0.5064
0.5797
0.6315
0.6764
0.7243
0.7859
0.8446
0.9042
0.9677
1.0
1.0

The printed values change, ranging from 0 when the position of the potentiometer is on one end to 1 when it’s on the other end. Note that these are float values, which may require conversion depending on the application.